In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("bucketing").getOrCreate()
print(spark)

25/03/17 16:52:32 WARN Utils: Your hostname, TTNPL-8203 resolves to a loopback address: 127.0.1.1; using 10.1.243.232 instead (on interface wlp0s20f3)
25/03/17 16:52:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 16:52:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/17 16:52:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Create a sample DataFrame
data = [
    (1, "Alice", 25),
    (2, "Bob", 30),
    (3, "Charlie", 28),
    (4, "David", 32),
    (5, "Eve", 27),
]
columns = ["id", "name", "age"]
df = spark.createDataFrame(data, columns)

In [5]:

# Register the DataFrame as a temporary view
df.createOrReplaceTempView("temp_table")

In [6]:

# Create a bucketed and sorted table using SQL
bucketed_path = "/home/hdoop/Docs/spark_files/17May2025/bucketing/bucket_data"
spark.sql(f"""
    CREATE TABLE bucketed_table
    (id INT, name STRING, age INT)
    USING parquet
    CLUSTERED BY (id) SORTED BY (age) INTO 4 BUCKETS
    LOCATION '{bucketed_path}'
""")


25/03/17 16:56:58 WARN HadoopFSUtils: The directory file:/home/hdoop/Docs/spark_files/17May2025/bucketing/bucket_data was not found. Was it deleted very recently?


DataFrame[]

In [7]:
# Insert data from the temporary view into the bucketed table
spark.sql("INSERT OVERWRITE TABLE bucketed_table SELECT * FROM temp_table")


25/03/17 16:57:15 WARN HadoopFSUtils: The directory file:/home/hdoop/Docs/spark_files/17May2025/bucketing/bucket_data was not found. Was it deleted very recently?
                                                                                

DataFrame[]

In [8]:
# Read the bucketed data
bucketed_df = spark.read.parquet(bucketed_path)
bucketed_df.show()


+---+-------+---+
| id|   name|age|
+---+-------+---+
|  3|Charlie| 28|
|  1|  Alice| 25|
|  4|  David| 32|
|  5|    Eve| 27|
|  2|    Bob| 30|
+---+-------+---+



In [9]:
spark.stop()